## CHEME 5660 Actual Prelim 2 Options Question

A trader at Olin Financial, an up-and-coming hedge fund, sold a short strangle on firm XYZ with a `01/20/2023` expiration. 

__Assumptions__: (i) the short put (contract 1) has a strike price of $K_{1}$ = 230.0 USD/share and an implied volatility of 58.97\%; (ii) the short call (contract 2) has a strike price of $K_{2}$ = 300.0 USD/share and an implied volatility of 52.59\%; (iii) there are 78 days to 01/20/2023 (from today); (iv) the current share price of XYZ is 270.89 USD/share; (v) the risk-free rate is 4.10\%.

Use the Jupyter notebook `CHEME-5660-PP2-Options.ipynb`, and any associated data sets or other course materials to answer the following questions:

* a) Compute the premiums for the put $\mathcal{P}_{1}$ and call $\mathcal{P}_{2}$ contracts for the `01/20/2023` short strangle on firm XYZ using the Cox, Ross, and Rubinstein (CRR) binomial lattice model. 
* b) Compute the maximum profit and break-even points for the Olin Financial short strangle position on XYZ.
* c) Compute the probability of the profit at expiration for the short strangle position by sampling the share price distribution from the `Equity` notebook. 

## Solution

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/prelims/P2/actual`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/prelims/P2/actual/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/prelims/P2/actual/Manifest.toml`


In [2]:
# load external packages that are required for the calculations -
using DataFrames
using CSV
using Dates
using Statistics
using LinearAlgebra
using Plots
using Colors
using Distributions
using StatsPlots
using PQEcolaPoint
using PrettyTables

# setup paths to load XYZ OHLC data set -
const _NOTEBOOK_ROOT = pwd();
const _PATH_TO_DATA = joinpath(_NOTEBOOK_ROOT, "data");

In [3]:
include("CHEME-5560-AP2-CodeLib.jl"); # Look inside me to find out what I have!

In [4]:
# Date -
D = Date(2022, 12, 16); # date when contracts expire 

# contract 1 parameters -
IV₁ = 58.97; # implied volatility for contract 1 (short put)
K₁ = 230.0;   # strike price short put
T₁ = ticker("P", "XYZ", D, K₁);

# contract 2 parameters -
IV₂ = 52.59; # implied volatility for contract 2 (short call)
K₂ = 300.50;  # strike price short call
T₂ = ticker("C", "XYZ", D, K₂);

# setup some shared constants
B = 365.0;   # number of days in a year
μ = 0.0410;  # risk-free rate
DTE = 78.0;  # days to expiration

# What is the current share price?
Sₒ = 270.89;

# How many levels on the tree?
L = 100;

# How many sample do we have?
number_of_samples = 10000;

### a) Estimate the price of the call and put contracts

In [5]:
# build the contracts for the trades - 

# Put contract -
xyz_put_contract_model = build(PutContractModel, (
    ticker = T₁,
    expiration_date = D,
    strike_price = K₁,
    premium = 0.0,
    number_of_contracts = 1,
    direction = 1, # Bug or feature? => always use 1 here (even if we are selling)
    current_price = 0.0      
));

# Call contract - 
xyz_call_contract_model = build(CallContractModel, (
    ticker = T₂,
    expiration_date = D,
    strike_price = K₂,
    premium = 0.0,
    number_of_contracts = 1,
    direction = 1, # Bug or feature? => always use 1 here (even if we are selling)
    current_price = 0.0      
));

In [6]:
# build lattice models -
my_put_lattice_model = build(CRRLatticeModel; number_of_levels=(L+1), Sₒ = Sₒ, σ = (IV₁/100), μ = μ, T = (DTE/B));
my_call_lattice_model = build(CRRLatticeModel; number_of_levels=(L+1), Sₒ = Sₒ, σ = (IV₂/100), μ = μ, T = (DTE/B));

In [7]:
# compute the price of the put contract using the premium method
P₁ = premium(xyz_put_contract_model, my_put_lattice_model);

In [8]:
# compute the price of the call contract using the premium method
P₂ = premium(xyz_call_contract_model, my_call_lattice_model);

In [17]:
# table for part A -
part_A_table_data = Array{Any, 2}(undef, 3,2);
part_A_table_data[1,1] = "Premium Short Put (contract 1)"
part_A_table_data[1,2] = P₁;

part_A_table_data[2,1] = "Premium Short Call (contract 2)"
part_A_table_data[2,2] = P₂

part_A_table_data[3,1] = "Total"
part_A_table_data[3,2] = P₁+P₂

# header part B -
header_table_part_A = (["Item", "USD/share"]);

# make table -
pretty_table(part_A_table_data; header = header_table_part_A)

┌─────────────────────────────────┬───────────┐
│                            Item │ USD/share │
├─────────────────────────────────┼───────────┤
│  Premium Short Put (contract 1) │   10.6792 │
│ Premium Short Call (contract 2) │   15.9402 │
│                           Total │   26.6194 │
└─────────────────────────────────┴───────────┘


### b) Compute the maximum profit and breakeven points for the short strangle at expiration

In [9]:
# max profit is the sum of the premiums (we make this when we open the trade)
max_profit_xyz_short_strangle = P₁+P₂;

In [10]:
# breakeven low B₁ -
B₁ = K₁ - max_profit_xyz_short_strangle;

In [11]:
# breakeven high B₂ -
B₂ = K₂ + max_profit_xyz_short_strangle;

In [16]:
# build a table with the max profit and break-even values -
part_B_table_data = Array{Any, 2}(undef, 3,2);
part_B_table_data[1,1] = "Max profit"
part_B_table_data[1,2] = max_profit_xyz_short_strangle;
part_B_table_data[2,1] = "B₁ (low break-even)"
part_B_table_data[2,2] = B₁
part_B_table_data[3,1] = "B₂ (high break-even)"
part_B_table_data[3,2] = B₂

# header part B -
header_table_part_B = (["Item", "USD/share"]);

# make table -
pretty_table(part_B_table_data; header = header_table_part_B)

┌──────────────────────┬───────────┐
│                 Item │ USD/share │
├──────────────────────┼───────────┤
│           Max profit │   26.6194 │
│  B₁ (low break-even) │   203.381 │
│ B₂ (high break-even) │   327.119 │
└──────────────────────┴───────────┘


### c) Compute the probability of profit at expiration 
The probability of profit for a short strangle is the probability that we close between the low and high break-even points. Thus, we are looking for the value:

$$P(B_{1}<X\leq{B_{2}}) = F_{X}(B_{2}) - F_{X}(B_{1})$$

where $B_{\star}$ denotes the respective break-even points, and $F_{X}(x)$ is:

$$F_{X}(x) = P(X\leq{x})$$

that is, the probability that a random-variable $X$ is less than (or equal) to a specified value $x$. Check out the `P` function in the CodeLib to estimate $P(X\leq{x})$ or to fit a cumulative distribution, see [the Distributions.jl documentation](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.cdf-Tuple{UnivariateDistribution,%20Real}).

In [18]:
# build a log normal distribution from the previous GBM simulations 

# from Q1
μ̂ = 5.108608304369271;
σ̂ = 0.3971754813129187;

# build -
d = LogNormal(μ̂, σ̂);

In [19]:
# sample the future price distribution -
pop_samples = rand(d, number_of_samples);

In [25]:
# compute the probability of profit -
P_B₁ = cdf(d, B₁) # probability that we will close *below* B1 - 

0.698415287032371

In [ ]:
P_B1 = cdf(d, B\_2) # probability that we will close *below* B1 - 

In [24]:
probability_of_profit = Fₓ_B₂ - Fₓ_B₁

0.259